In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.3.1               |             py_0          25 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-3.1.0               |           py36_0         724 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following NEW packages will be 

In [5]:
df = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## DATA PREPARATION

In [8]:
df = df [df.Borough != "Not assigned"]
df.shape

(211, 3)

In [9]:
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [11]:
df_canada = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df_canada.shape

(103, 3)

In [12]:
df_canada.loc[df_canada['Neighbourhood']=="Not assigned",'Neighbourhood'] = df_canada['Borough']

In [19]:
df_canada 

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


## GEODATA

In [13]:
!wget -O Geospatial_Coordinates.csv http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv

--2019-07-30 02:02:06--  http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv
Resolving cocl.us (cocl.us)... 169.48.113.201
Connecting to cocl.us (cocl.us)|169.48.113.201|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv [following]
--2019-07-30 02:02:06--  https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv
Connecting to cocl.us (cocl.us)|169.48.113.201|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-07-30 02:02:09--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.26.197
Connecting to ibm.box.com (ibm.box.com)|107.152.26.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following

In [14]:
geo_df = pd.read_csv("Geospatial_Coordinates.csv")
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
geo_df = geo_df.rename({'Postal Code':'Postcode'}, axis = 1)
geo_df.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
df_cageo = pd.merge(df_canada, geo_df, on='Postcode', how = 'left')
df_cageo.shape

(103, 5)

In [17]:
df_cageo

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


## TORONTO DATA

In [18]:
df_toronto = df_cageo[df_cageo['Borough'].str.contains('Toronto')]
df_toronto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [19]:
df_toronto.reset_index(drop=True,inplace = True)  #relabel the index
df_toronto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [20]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [21]:
map_toronto = folium.Map(location = [latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)


map_toronto

## VENUE DATA

In [22]:
CLIENT_ID = 'GIRPXY04BYSD0OWJSRE0CLICRIXKGICEOADZWT5QG5IX0TIR' # your Foursquare ID
CLIENT_SECRET = 'MJURV5NDEMJXFOXWJ2TOO0GOKQCEVIH2TEYS2PVL3SB5CYF3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GIRPXY04BYSD0OWJSRE0CLICRIXKGICEOADZWT5QG5IX0TIR
CLIENT_SECRET:MJURV5NDEMJXFOXWJ2TOO0GOKQCEVIH2TEYS2PVL3SB5CYF3


In [23]:
# Get data of first neighborhood
neighborhood_latitude = df_toronto['Latitude'][0] # neighborhood latitude value
neighborhood_longitude = df_toronto['Longitude'][0] # neighborhood longitude value

neighborhood_name = df_toronto['Neighbourhood'][0] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


In [24]:
search_query = 'coffee'
radius = 500
LIMIT =300
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url


'https://api.foursquare.com/v2/venues/search?client_id=GIRPXY04BYSD0OWJSRE0CLICRIXKGICEOADZWT5QG5IX0TIR&client_secret=MJURV5NDEMJXFOXWJ2TOO0GOKQCEVIH2TEYS2PVL3SB5CYF3&ll=43.653963,-79.387207&v=20180605&query=coffee&radius=500&limit=300'

In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, search_query, radius, LIMIT)
        print(url)
        # make the GET request
        results = requests.get(url).json()["response"]['venues']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['name'], 
            v['location']['lat'], 
            v['location']['lng']) for v in results])
        # return only relevant information for each nearby venue

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude']
    
    return(nearby_venues)

In [26]:
#Get all Tor
df_toronto_venues = getNearbyVenues(names=df_toronto['Neighbourhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

The Beaches
https://api.foursquare.com/v2/venues/search?client_id=GIRPXY04BYSD0OWJSRE0CLICRIXKGICEOADZWT5QG5IX0TIR&client_secret=MJURV5NDEMJXFOXWJ2TOO0GOKQCEVIH2TEYS2PVL3SB5CYF3&ll=43.67635739999999,-79.2930312&v=20180605&query=coffee&radius=500&limit=300
The Danforth West, Riverdale
https://api.foursquare.com/v2/venues/search?client_id=GIRPXY04BYSD0OWJSRE0CLICRIXKGICEOADZWT5QG5IX0TIR&client_secret=MJURV5NDEMJXFOXWJ2TOO0GOKQCEVIH2TEYS2PVL3SB5CYF3&ll=43.6795571,-79.352188&v=20180605&query=coffee&radius=500&limit=300
The Beaches West, India Bazaar
https://api.foursquare.com/v2/venues/search?client_id=GIRPXY04BYSD0OWJSRE0CLICRIXKGICEOADZWT5QG5IX0TIR&client_secret=MJURV5NDEMJXFOXWJ2TOO0GOKQCEVIH2TEYS2PVL3SB5CYF3&ll=43.6689985,-79.31557159999998&v=20180605&query=coffee&radius=500&limit=300
Studio District
https://api.foursquare.com/v2/venues/search?client_id=GIRPXY04BYSD0OWJSRE0CLICRIXKGICEOADZWT5QG5IX0TIR&client_secret=MJURV5NDEMJXFOXWJ2TOO0GOKQCEVIH2TEYS2PVL3SB5CYF3&ll=43.6595255,-79.3409

In [29]:
print(df_toronto_venues.shape)
df_toronto_venues

(387, 6)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
0,The Beaches,43.676357,-79.293031,Dvine Gelato Lounge & Coffee Bar,43.672500,-79.288530
1,The Beaches,43.676357,-79.293031,Oscar Coffee and Espresso Bar,43.672600,-79.287701
2,The Beaches,43.676357,-79.293031,Best Coffee House,43.672494,-79.287690
3,"The Danforth West, Riverdale",43.679557,-79.352188,Timothy's World Coffee,43.677581,-79.353199
4,"The Danforth West, Riverdale",43.679557,-79.352188,Marvel Coffee Co.,43.678630,-79.347460
5,"The Danforth West, Riverdale",43.679557,-79.352188,Starbucks,43.678730,-79.346188
6,"The Beaches West, India Bazaar",43.668999,-79.315572,Coffee Time,43.667814,-79.311798
7,"The Beaches West, India Bazaar",43.668999,-79.315572,Coffee time,43.673001,-79.319305
8,"The Beaches West, India Bazaar",43.668999,-79.315572,unicorn coffee,43.672674,-79.320412
9,Studio District,43.659526,-79.340923,First Break Coffee Shop,43.663456,-79.341063


In [30]:
df_cg=df_toronto_venues.groupby('Neighborhood').count()
df_cg

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Neighborhood,,,,,
"Adelaide, King, Richmond",41,41,41,41,41
Berczy Park,14,14,14,14,14
"Brockton, Exhibition Place, Parkdale Village",6,6,6,6,6
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",1,1,1,1,1
"Cabbagetown, St. James Town",1,1,1,1,1
Central Bay Street,30,30,30,30,30
"Chinatown, Grange Park, Kensington Market",11,11,11,11,11
Church and Wellesley,10,10,10,10,10
"Commerce Court, Victoria Hotel",38,38,38,38,38


In [31]:
MaxValue=df_cg['Venue'].max()
MaxValue

41

In [32]:
CList = df_cg['Venue'].tolist()
CList[:] = [(x / MaxValue).round(2) for x in CList]
CList

[1.0,
 0.34,
 0.15,
 0.02,
 0.02,
 0.73,
 0.27,
 0.24,
 0.93,
 0.1,
 0.02,
 0.02,
 0.83,
 0.07,
 0.98,
 0.12,
 0.32,
 0.29,
 0.02,
 0.29,
 0.02,
 0.05,
 0.02,
 0.17,
 0.68,
 0.63,
 0.61,
 0.1,
 0.15,
 0.07,
 0.07,
 0.07]

In [33]:
df_cg['Weight'] = CList 

In [34]:
df_cg

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Weight
Neighborhood,,,,,,
"Adelaide, King, Richmond",41,41,41,41,41,1.00
Berczy Park,14,14,14,14,14,0.34
"Brockton, Exhibition Place, Parkdale Village",6,6,6,6,6,0.15
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",1,1,1,1,1,0.02
"Cabbagetown, St. James Town",1,1,1,1,1,0.02
Central Bay Street,30,30,30,30,30,0.73
"Chinatown, Grange Park, Kensington Market",11,11,11,11,11,0.27
Church and Wellesley,10,10,10,10,10,0.24
"Commerce Court, Victoria Hotel",38,38,38,38,38,0.93


In [35]:
df_cor=df_toronto_venues.groupby('Neighborhood').mean().reset_index()
df_cor['Weight'] = CList 
df_cor

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Longitude,Weight
0,"Adelaide, King, Richmond",43.650571,-79.384568,43.650569,-79.384867,1.00
1,Berczy Park,43.644771,-79.373306,43.645972,-79.375676,0.34
2,"Brockton, Exhibition Place, Parkdale Village",43.636847,-79.428191,43.638813,-79.427204,0.15
3,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420,43.631392,-79.395952,0.02
4,"Cabbagetown, St. James Town",43.667967,-79.367675,43.665295,-79.368335,0.02
5,Central Bay Street,43.657952,-79.387383,43.657421,-79.386980,0.73
6,"Chinatown, Grange Park, Kensington Market",43.653206,-79.400049,43.654555,-79.401049,0.27
7,Church and Wellesley,43.665860,-79.383160,43.665944,-79.384018,0.24
8,"Commerce Court, Victoria Hotel",43.648199,-79.379817,43.648932,-79.380793,0.93
9,Davisville,43.704324,-79.388790,43.706139,-79.389168,0.10


In [41]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

In [54]:
from folium import plugins
# let's start again with a clean copy of the map of San Francisco
map_clusters = folium.Map(location = [latitude, longitude], zoom_start=12)

for lat, lng, label,weight in zip(df_cor['Neighborhood Latitude'], df_cor['Neighborhood Longitude'], df_cor['Neighborhood'], df_cor['Weight']):
    fcolor =plt.cm.Reds(weight*100)
    fcolor = mpl.colors.to_hex(fcolor)
    folium.features.CircleMarker(
        [lat, lng],
        radius=(weight*100%10)*2, # define how big you want the circle markers to be
        colour=fcolor,
        fill=True,
        popup=label,
        fill_color=fcolor,
        fill_opacity=1
    ).add_to(map_clusters)

# display map
map_clusters